# <center>Lab Assignment 3: Hopfield networks</center>

In [1]:
import numpy as np

### Implement the network class

In [2]:
class HopfieldNetwork:
    def __init__(self):
        self.W = None

    def train(self, X: np.ndarray):
        M, N = len(X), len(X[0])
        self.W = (1 / N) * X.dot(X.T)
        # np.fill_diagonal(self.W, 0)

    def print_W(self):
        for i in range(self.W.shape[0]):
            for j in range(self.W.shape[1]):
                print("%.1f  \t" % (self.W[i][j]),  end='')
            print()

    def update(self, V):
        """
        Apply the update rule on the input pattern V to get the restored pattern
        which is an attractor in the network's storage
        :param V: the input pattern
        :return: the restored pattern
        """
        m = len(V)
        indices = [i for i in range(m)]
        iter = 1
        while True:
            cnt = 0
            indices = np.random.permutation(indices)
            for i in indices:
                value_old = V[i]
                value_new = np.sign(self.W[:, i].dot(V))
                if value_new != value_old:
                    V[i] = value_new
                    cnt += 1
                # print(i, value_old, value_new, self.W[:, i].dot(V))
            if cnt == 0:
                # print("Converged in %s iterations" % iter)
                break
            iter += 1

        return V

### Construct input patterns for training

In [3]:
x1 = [-1, -1, 1, -1, 1, -1, -1, 1]
x2 = [-1, -1, -1, -1, -1, 1, -1, -1]
x3 = [-1, 1, 1, -1, -1, 1, -1, 1]
X = np.array([x1, x2, x3]).T

### Train the network

In [4]:
model = HopfieldNetwork()
model.train(X)

# 3. Tasks and questions

## 3.1 Convergence and attractors

In [5]:
# 1. apply the update rule
x1d = np.array([1, -1, 1, -1, 1, -1, -1, 1])
x2d = np.array([1, 1, -1, -1, -1, 1, -1, -1])
x3d = np.array([1, 1, 1, -1, 1, 1, -1, 1])

In [6]:
rp1 = model.update(x1d)
print(rp1)
print((rp1 == x1).all())            

[-1 -1  1 -1  1 -1 -1  1]
True


In [7]:
rp2 = model.update(x2d)
print(rp2)
print((rp2 == x2).all())

[-1 -1 -1 -1 -1  1 -1 -1]
True


In [8]:
rp3 = model.update(x3d)
print(rp3)
print((rp3 == x3).all())

[-1  1  1 -1  1 -1 -1  1]
False


In [9]:
# 2. How many attractors are there in this network? Answer: 14
N = 8
k = 2 ** N
# print('2^N=', k)
rp = np.ones([k, N])

for i in range(k):
    l = len(bin(i)) - 2
    for j in range(l):
        if bin(i)[j + 2] == '1':
            rp[i, N - l + j] = -1

n = 0

for i in range(k):
#    print(i, ':', rp[i])
    A = np.copy(rp[i])
    B = model.update(rp[i])
#    print(rp[i])
#    print(A)
#    print(B)
#    print((A==B).all())
    if ((A==B).all())==True:
        n+=1
        print(n,':',A)

1 : [ 1.  1.  1.  1.  1. -1.  1.  1.]
2 : [ 1.  1.  1.  1. -1.  1.  1.  1.]
3 : [ 1.  1. -1.  1.  1. -1.  1. -1.]
4 : [ 1.  1. -1.  1. -1.  1.  1. -1.]
5 : [ 1. -1. -1.  1.  1. -1.  1. -1.]
6 : [ 1. -1. -1.  1. -1.  1.  1. -1.]
7 : [-1.  1.  1. -1.  1. -1. -1.  1.]
8 : [-1.  1.  1. -1. -1.  1. -1.  1.]
9 : [-1. -1.  1. -1.  1. -1. -1.  1.]
10 : [-1. -1.  1. -1. -1.  1. -1.  1.]
11 : [-1. -1. -1. -1.  1. -1. -1. -1.]
12 : [-1. -1. -1. -1. -1.  1. -1. -1.]


In [10]:
# 3. What happens when making the start pattern more dissimilar to the stored ones?

In [11]:
x3ds = np.array([1, -1, -1, 1, 1, 1, -1, 1])
r3ds = model.update(x3ds)
print(r3ds)
print((r3ds == x3).all())

[ 1 -1 -1  1  1 -1  1 -1]
False


## 3.2 Sequential update